<a href="https://colab.research.google.com/github/ashimasinghal/Indian-NER/blob/main/tamil_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seqeval
!pip install transformers

     |████████████████████████████████| 43 kB 1.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=162ec3f668d378f47b001a998f91e6d5014427f061e8cb489263da2a42be4499
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 35.0 MB/s 
     |████████████████████████████████| 895 kB 33.2 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 3.3 MB 31.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

In [ ]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [ ]:
!pip list | grep -E 'transformers|torch|Keras'

Keras-Preprocessing           1.1.2
torch                         1.10.0+cu111
torchaudio                    0.10.0+cu111
torchsummary                  1.5.1
torchtext                     0.11.0
torchvision                   0.11.1+cu111
transformers                  4.12.5


In [ ]:
# load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BERT/NER Corpus/Tamil.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df

,Unnamed: 0,word,T1,T2,T3,T4,T5
0,0,',SYM,I-NP,o,o,o
1,1,'அடிமைமுறை,NNP,I-NP,o,o,o
2,2,',SYM,I-NP,o,o,o
3,3,',SYM,I-NP,o,o,o
4,4,',SYM,I-NP,o,o,o
...,...,...,...,...,...,...,...
94306,94306,லித்தியத்தின்,NN,B-NP,B-MATERIALS,B-CHEMICAL,o
94307,94307,செழுமையை,NN,B-NP,o,o,o
94308,94308,மதிப்பிட,VM,B-VGNF,o,o,o
94309,94309,முடியும்,VAUX,B-VGF,o,o,o


In [ ]:
sentence_id = []
words = df['word']
sentence = 1
for word in words:
  sentence_id.append(sentence)
  if(word=='.'):
    sentence += 1

In [ ]:
df.insert(1,'Sentence ID',sentence_id,allow_duplicates=True)
df.head(50)

,Unnamed: 0,Sentence ID,word,T1,T2,T3,T4,T5
0,0,1,',SYM,I-NP,o,o,o
1,1,1,'அடிமைமுறை,NNP,I-NP,o,o,o
2,2,1,',SYM,I-NP,o,o,o
3,3,1,',SYM,I-NP,o,o,o
4,4,1,',SYM,I-NP,o,o,o
5,5,1,என்பது,UT,B-BLK,o,o,o
6,6,1,வலுக்கட்டாயமாக,RB,B-RBP,o,o,o
7,7,1,மனிதர்களைப்,NN,B-NP,o,o,o
8,8,1,பிற,JJ,B-NP,o,o,o
9,9,1,மனிதர்கள்,NN,I-NP,o,o,o


In [ ]:
hindi = pd.DataFrame()
hindi = df[['Sentence ID','word','T2','T3']]
hindi.rename(columns={'Sentence ID': 'Sentence', 'T2': 'POS','T3':'TAG'}, inplace=True)
hindi

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Sentence,word,POS,TAG
0,1,',I-NP,o
1,1,'அடிமைமுறை,I-NP,o
2,1,',I-NP,o
3,1,',I-NP,o
4,1,',I-NP,o
...,...,...,...,...
94306,6345,லித்தியத்தின்,B-NP,B-MATERIALS
94307,6345,செழுமையை,B-NP,o
94308,6345,மதிப்பிட,B-VGNF,o
94309,6345,முடியும்,B-VGF,o


In [ ]:
hindi.POS.unique()

array(['I-NP', 'B-BLK', 'B-RBP', 'B-NP', 'B-VGNF', 'I-VGNF', 'B-VGF',
       'I-VGF', 'I-RBP', 'o', 'B-VGINF', 'B-NEGP', 'I-NEGP', 'B-VGNN',
       'I-VGNN', 'B-JJP', 'B-CCP', 'I-CCP', 'I-BLK', 'NP', 'I-JJP',
       'I-VGINF', 'VGF'], dtype=object)

In [ ]:
hindi.TAG.unique()

array(['o', 'B-PERIOD', 'I-PERIOD', 'B-COUNT', 'B-LOCATION', 'B-PERSON',
       'B-YEAR', 'I-PERSON', 'I-COUNT', 'B-ORGANIZATION',
       'I-ORGANIZATION', 'I-YEAR', 'I-LOCATION', 'B-DISTANCE',
       'B-ARTIFACT', 'B-PLANTS', 'B-MATERIALS', 'B-LOCOMOTIVE',
       'B-QUANTITY', 'I-QUANTITY', 'B-MONEY', 'I-MONEY', 'I-ARTIFACT',
       'B-FACILITIES', 'I-FACILITIES', 'B-LIVTHINGS', 'I-DISTANCE',
       'B-MONTH', 'I-MONTH', 'B-ENTERTAINMENT', 'I-ENTERTAINMENT',
       'B-SDAY', 'B-TIME', 'I-TIME', 'B-DATE', 'I-DATE', 'B-DAY',
       'I-PLANTS', 'I-SDAY', 'I-DAY', 'I-MATERIALS', 'B-DISEASE',
       'I-DISEASE', 'I-LIVTHINGS', 'B-ENAMEX', 'I-ENAMEX', 'I-LOCOMOTIVE'],
      dtype=object)

In [ ]:
hindi['Sentence'].nunique(), hindi.word.nunique(), hindi.POS.nunique(), hindi.TAG.nunique()

(6345, 31265, 23, 47)

In [ ]:

hindi.TAG.value_counts()

o                  83241
B-PERSON            1985
B-LOCATION          1774
B-COUNT             1086
I-PERSON             633
B-YEAR               567
I-YEAR               565
B-MATERIALS          522
B-ARTIFACT           452
I-LOCATION           428
B-LIVTHINGS          302
I-PERIOD             238
B-ORGANIZATION       211
B-MONTH              205
I-ORGANIZATION       155
I-ARTIFACT           148
B-PERIOD             134
I-TIME               125
I-LIVTHINGS          114
I-COUNT              110
I-DATE               100
I-MATERIALS           88
B-LOCOMOTIVE          86
B-TIME                83
I-DISTANCE            82
I-QUANTITY            80
B-PLANTS              78
B-QUANTITY            74
B-ENTERTAINMENT       74
I-ENTERTAINMENT       71
I-FACILITIES          67
B-DATE                60
B-DISTANCE            58
I-MONTH               51
B-FACILITIES          38
I-LOCOMOTIVE          37
B-DAY                 32
I-MONEY               24
B-DISEASE             24
B-MONEY               23


In [ ]:
# parsing the data


In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# hindi_train, hindi_test = train_test_split( hindi, test_size=0.33, random_state=42)

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["TAG"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(hindi)

In [ ]:
sentences = [[s[0] for s in sent] for sent in getter.sentences]
sentences[0]

["'",
 "'அடிமைமுறை",
 "'",
 "'",
 "'",
 'என்பது',
 'வலுக்கட்டாயமாக',
 'மனிதர்களைப்',
 'பிற',
 'மனிதர்கள்',
 'பிடித்து',
 'வைத்து',
 ',',
 'அவர்களிடமிருந்து',
 'வேலையை',
 'கட்டாயமாக',
 'வாங்குவதாகும்',
 '.']

In [ ]:
poses = [[s[1] for s in sent] for sent in getter.sentences]
print(poses[0])

['I-NP', 'I-NP', 'I-NP', 'I-NP', 'I-NP', 'B-BLK', 'B-RBP', 'B-NP', 'B-NP', 'I-NP', 'B-VGNF', 'B-VGNF', 'I-VGNF', 'B-NP', 'B-NP', 'B-RBP', 'B-VGF', 'I-VGF']


In [ ]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])

['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']


In [ ]:
# Make TAG name into index for training
tags_vals = list(set(hindi["TAG"].values))
tags_vals.append('X')
tags_vals.append('[CLS]')
tags_vals.append('[SEP]')
tags_vals = set(tags_vals)
tags_vals

{'B-ARTIFACT',
 'B-COUNT',
 'B-DATE',
 'B-DAY',
 'B-DISEASE',
 'B-DISTANCE',
 'B-ENAMEX',
 'B-ENTERTAINMENT',
 'B-FACILITIES',
 'B-LIVTHINGS',
 'B-LOCATION',
 'B-LOCOMOTIVE',
 'B-MATERIALS',
 'B-MONEY',
 'B-MONTH',
 'B-ORGANIZATION',
 'B-PERIOD',
 'B-PERSON',
 'B-PLANTS',
 'B-QUANTITY',
 'B-SDAY',
 'B-TIME',
 'B-YEAR',
 'I-ARTIFACT',
 'I-COUNT',
 'I-DATE',
 'I-DAY',
 'I-DISEASE',
 'I-DISTANCE',
 'I-ENAMEX',
 'I-ENTERTAINMENT',
 'I-FACILITIES',
 'I-LIVTHINGS',
 'I-LOCATION',
 'I-LOCOMOTIVE',
 'I-MATERIALS',
 'I-MONEY',
 'I-MONTH',
 'I-ORGANIZATION',
 'I-PERIOD',
 'I-PERSON',
 'I-PLANTS',
 'I-QUANTITY',
 'I-SDAY',
 'I-TIME',
 'I-YEAR',
 'X',
 '[CLS]',
 '[SEP]',
 'o'}

In [ ]:
tag2idx={'B-ARTIFACT':1,
 'B-COUNT':2,
 'B-DATE':3,
 'B-DAY':4,
 'B-DISEASE':5,
 'B-DISTANCE':6,
 'B-ENAMEX':7,
 'B-ENTERTAINMENT':8,
 'B-FACILITIES':9,
 'B-LIVTHINGS':10,
 'B-LOCATION':11,
 'B-LOCOMOTIVE':12,
 'B-MATERIALS':13,
 'B-MONEY':14,
 'B-MONTH':15,
 'B-ORGANIZATION':16,
 'B-PERIOD':17,
 'B-PERSON':18,
 'B-PLANTS':19,
 'B-QUANTITY':20,
 'B-SDAY':21,
 'B-TIME':22,
 'B-YEAR':23,
 'I-ARTIFACT':24,
 'I-COUNT':25,
 'I-DATE':26,
 'I-DAY':27,
 'I-DISEASE':28,
 'I-DISTANCE':29,
 'I-ENAMEX':30,
 'I-ENTERTAINMENT':31,
 'I-FACILITIES':32,
 'I-LIVTHINGS':33,
 'I-LOCATION':34,
 'I-LOCOMOTIVE':35,
 'I-MATERIALS':36,
 'I-MONEY':37,
 'I-MONTH':38,
 'I-ORGANIZATION':39,
 'I-PERIOD':40,
 'I-PERSON':41,
 'I-PLANTS':42,
 'I-QUANTITY':43,
 'I-SDAY':44,
 'I-TIME':45,
 'I-YEAR':46,
 'X':47,
 '[CLS]':48,
 '[SEP]':49,
 'o':50
 
 }
tag2idx

{'B-ARTIFACT': 1,
 'B-COUNT': 2,
 'B-DATE': 3,
 'B-DAY': 4,
 'B-DISEASE': 5,
 'B-DISTANCE': 6,
 'B-ENAMEX': 7,
 'B-ENTERTAINMENT': 8,
 'B-FACILITIES': 9,
 'B-LIVTHINGS': 10,
 'B-LOCATION': 11,
 'B-LOCOMOTIVE': 12,
 'B-MATERIALS': 13,
 'B-MONEY': 14,
 'B-MONTH': 15,
 'B-ORGANIZATION': 16,
 'B-PERIOD': 17,
 'B-PERSON': 18,
 'B-PLANTS': 19,
 'B-QUANTITY': 20,
 'B-SDAY': 21,
 'B-TIME': 22,
 'B-YEAR': 23,
 'I-ARTIFACT': 24,
 'I-COUNT': 25,
 'I-DATE': 26,
 'I-DAY': 27,
 'I-DISEASE': 28,
 'I-DISTANCE': 29,
 'I-ENAMEX': 30,
 'I-ENTERTAINMENT': 31,
 'I-FACILITIES': 32,
 'I-LIVTHINGS': 33,
 'I-LOCATION': 34,
 'I-LOCOMOTIVE': 35,
 'I-MATERIALS': 36,
 'I-MONEY': 37,
 'I-MONTH': 38,
 'I-ORGANIZATION': 39,
 'I-PERIOD': 40,
 'I-PERSON': 41,
 'I-PLANTS': 42,
 'I-QUANTITY': 43,
 'I-SDAY': 44,
 'I-TIME': 45,
 'I-YEAR': 46,
 'X': 47,
 '[CLS]': 48,
 '[SEP]': 49,
 'o': 50}

In [ ]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [ ]:
# training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

1

In [ ]:
# vocabulary = "/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased/bert-base-multilingual-cased-vocab.txt"

In [ ]:
max_len  = 45
# tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased', truncation=True, do_lower_case=True)
# tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
# model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("skillzzzzzy/tamilberto")

Downloading:   0%|          | 0.00/671 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/530k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331k [00:00<?, ?B/s]

In [ ]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1

No.0,len:100
texts:[CLS] ' ' à®ħà®Ł à®¿ à®® à¯Ī à®® à¯ģ à®± à¯Ī ' ' ' à®İà®© à¯į à®ªà®¤ à¯ģ à®µà®² à¯ģ à®ķ à¯į à®ķà®Ł à¯į à®Ł à®¾ à®¯à®® à®¾ à®ķ à®®à®© à®¿ à®¤à®° à¯į à®ķà®³ à¯Ī à®ª à¯į à®ª à®¿ à®± à®®à®© à®¿ à®¤à®° à¯į à®ķà®³ à¯į à®ª à®¿ à®Ł à®¿ à®¤ à¯į à®¤ à¯ģ à®µ à¯Ī à®¤ à¯į à®¤ à¯ģ , à®ħà®µà®° à¯į à®ķà®³ à®¿ à®Łà®® à®¿ à®° à¯ģ à®¨ à¯į à®¤ à¯ģ à®µ à¯ĩ à®² à¯Ī à®¯ à¯Ī à®ķà®Ł à¯į à®Ł à®¾ à®¯à®® à®¾ à®ķ à®µ à®¾ à®Ļ à¯į à®ķ à¯ģ à®µà®¤ à®¾ à®ķ à¯ģ à®® à¯į . [SEP]
No.0,len:100
lables:[CLS] o o X X X X X X X X o o o o X X X o X X X X X X X X X X o X X X X X X X o X X o X X X X X o X X X X X X X o X X X X X o o X X X X X X X X X X X o X X X X X o X X X X X X o X X X X X X X X X X X o [SEP]
No.1,len:49
texts:[CLS] à®ĩà®® à¯į à®® à¯ģ à®± à¯Ī à®¨ à¯Ĩ à®Ł à¯ģ à®Ļ à¯į à®ķ à®¾ à®²à®® à®¾ à®ķ à®ªà®² à®¨ à®¾ à®Ł à¯ģ à®ķà®³ à®¿ à®² à¯į à®µà®´à®ķ à¯į à®ķ à®¿ à®² à¯į à®ĩà®° à¯ģ à®¨ à¯į à®¤ à¯ģ à®µà®¨ à¯į à®¤ à¯ģ à®³ à¯į à®³à®¤ à¯ģ . [SEP]
No.1,len:49
lables:[CLS] o X X X X X o X X X X X X X X X X o o 

In [ ]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[   3   11   11 1350  266  273  276  273  265  279  276   11   11   11
  721  263  316  265  460  265  267  263  472  263  271  270  347  270
  267  453  266  382  263  285  276  275  263  275  266  279  453  266
  382  263  285]


In [ ]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["o"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])


[48 50 50 47 47 47 47 47 47 47 47 50 50 50 50 47 47 47 50 47 47 47 47 47
 47 47 47 47 47 50 47 47 47 47 47 47 47 50 47 47 50 47 47 47 47]


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.3)

In [ ]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(4441, 1904, 4441, 1904)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [ ]:
batch_num = 32

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [ ]:
# model_file_address = '/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased'
# model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased",num_labels=len(tag2idx))
model = AutoModelForMaskedLM.from_pretrained("skillzzzzzy/tamilberto")


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

In [ ]:
m_labels = len(tag2idx)

In [ ]:
model;

In [ ]:
model.cuda();

In [ ]:
epochs = 30
max_grad_norm = 1.0

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [ ]:
# fine tuning
FULL_FINETUNING = True

In [ ]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
model.train();

In [ ]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

***** Running training *****
  Num examples = 4441
  Batch size = 32
  Num steps = 2780


Epoch:   5%|▌         | 1/20 [00:57<18:04, 57.07s/it]

Train loss: 6.059854752775552


Epoch:  10%|█         | 2/20 [01:54<17:05, 57.00s/it]

Train loss: 5.6594799843387325


...

Train loss: 0.8093175296334253


Epoch:  95%|█████████▌| 19/20 [18:03<00:57, 57.04s/it]

Train loss: 0.8087068619071573


Epoch: 100%|██████████| 20/20 [19:00<00:00, 57.04s/it]

Train loss: 0.8089000595652539


In [ ]:
bert_out_address = 'ashiNLP/bert_tamilberto_model/tamil'

In [ ]:
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)

In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model

In [ ]:
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

In [ ]:
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

('ashiNLP/bert_tamilberto_model/tamil/vocab.json',
 'ashiNLP/bert_tamilberto_model/tamil/merges.txt')

In [ ]:
# load model
model = BertForTokenClassification.from_pretrained(bert_out_address,num_labels=len(tag2idx))

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ashiNLP/bert_tamilberto_model/tamil were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.1.attention.self.key.weight', 'roberta.embeddings.position_ids', 'roberta.encoder.layer.1.attention.self.value.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.encoder.layer.2.intermediate.dense.bias', 'roberta.encoder.layer.4.attention.self.value.weight', 'roberta.encoder.layer.4.intermediate.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'roberta.embeddings.word_embeddings.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.2.attention.self.key.bias', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.5.at

In [ ]:
model.cuda();

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
# EVALUATION

In [ ]:
model.eval();

In [ ]:

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
# output_eval_file = os.path.join('/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased', "eval_results.txt")
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =1904
  Batch size = 32


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: o seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [CLS] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


f1 socre: 0.002565
Accuracy score: 0.001251


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** Eval results *****

               precision    recall  f1-score   support

     ARTIFACT     0.0000    0.0000    0.0000       104
         CLS]     0.0000    0.0000    0.0000         0
        COUNT     0.0000    0.0000    0.0000       201
         DATE     0.0000    0.0000    0.0000        17
          DAY     0.0000    0.0000    0.0000         3
      DISEASE     0.0000    0.0000    0.0000        12
     DISTANCE     0.0000    0.0000    0.0000        12
       ENAMEX     0.0000    0.0000    0.0000         0
ENTERTAINMENT     0.0000    0.0000    0.0000         9
   FACILITIES     0.0111    0.2000    0.0211         5
    LIVTHINGS     0.0027    0.0429    0.0050        70
     LOCATION     0.0258    0.0122    0.0166       328
   LOCOMOTIVE     0.0005    0.0769    0.0011        13
    MATERIALS     0.0124    0.0989    0.0221        91
        MONEY     0.0000    0.0000    0.0000         5
        MONTH     0.0022    0.0556    0.0043        36
 ORGANIZATION     0.0000    0.0000    

In [ ]:
save_model_address = 'ashiNLP/bert_multilingual_model/hindi'

In [ ]:
save_model = BertForTokenClassification.from_pretrained(save_model_address,num_labels=len(tag2idx))

In [ ]:
tokenizer = BertTokenizer.from_pretrained(save_model_address,do_lower_case=False)

In [ ]:
# predict
test_query = "मैं 20 साल से दिल्ली में रहता हूं"

In [ ]:
tokenized_texts = []

temp_token = []
temp_token.append('[CLS]')
token_list = tokenizer.tokenize(test_query)

token_list

['म', '##ैं', '20', 'साल', 'से', 'दिल्ली', 'में', 'रहता', 'ह', '##ू', '##ं']

In [ ]:
for m,token in enumerate(token_list):
    temp_token.append(token)

In [ ]:
if len(temp_token) > max_len-1:
    temp_token= temp_token[:max_len-1]

In [ ]:
temp_token.append('[SEP]')
temp_token

['[CLS]',
 'म',
 '##ैं',
 '20',
 'साल',
 'से',
 'दिल्ली',
 'में',
 'रहता',
 'ह',
 '##ू',
 '##ं',
 '[SEP]']

In [ ]:
tokenized_texts.append(temp_token)

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])


[  101   889 99007 10197 53749 11072 44293 10532 90213   899 15778 14018
   102     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0]


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:

segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];


In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
segment_ids = torch.tensor(segment_ids)

In [ ]:
model.eval();

In [ ]:
with torch.no_grad():
        outputs = save_model(input_ids, token_type_ids=None,
        attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 

In [ ]:
predict_results = logits.detach().cpu().numpy()
predict_results.shape


(1, 45, 48)

In [ ]:
from scipy.special import softmax
result_arrays_soft = softmax(predict_results[0])
result_arrays_soft[0]


array([7.8821239e-07, 1.6101421e-07, 2.6199646e-07, 2.8371966e-07,
       6.1914130e-07, 3.2382064e-07, 1.4583509e-07, 3.3334899e-07,
       2.4013846e-07, 6.4514126e-07, 1.8731113e-07, 2.2766055e-07,
       3.0222401e-07, 4.9296852e-07, 3.7287452e-07, 2.1302953e-06,
       7.7263093e-07, 1.1945436e-07, 1.0083087e-07, 4.4969744e-07,
       5.6775457e-07, 4.4234781e-07, 5.4370292e-07, 2.9001552e-07,
       2.0586785e-07, 2.0982175e-07, 3.9802552e-07, 3.3990912e-07,
       1.7846563e-07, 3.3698811e-07, 4.6914303e-07, 2.2363588e-07,
       3.5271682e-07, 1.7047587e-07, 2.4020099e-07, 2.9659734e-07,
       9.6733538e-07, 2.8670701e-07, 5.0679699e-07, 4.0585962e-07,
       1.8743552e-07, 1.0702756e-05, 1.5184305e-05, 2.7657900e-02,
       1.9282452e-06, 1.7326340e-07, 3.2972500e-07, 3.1048972e-07],
      dtype=float32)

In [ ]:
result_array = result_arrays_soft
len(result_array),len(result_array[0])

(45, 48)

In [ ]:
# Get each token final predict tag index result
result_list = np.argmax(result_array,axis=-1)
result_list

array([43, 42, 41, 14, 13, 42, 42, 42, 42, 42, 41, 41, 42, 42, 42, 42, 42,
       42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42,
       42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42])

In [ ]:
for i, mark in enumerate(attention_masks[0]):
    if mark>0:
        print("Token:%s"%(temp_token[i]))
#         print("Tag:%s"%(result_list[i]))
        print("Predict_Tag:%s"%(tag2name[result_list[i]]))
        #print("Posibility:%f"%(result_array[i][result_list[i]]))
        print()

Token:[CLS]
Predict_Tag:[CLS]

Token:म
Predict_Tag:o

Token:##ैं
Predict_Tag:X

Token:20
Predict_Tag:B-PERIOD

Token:साल
Predict_Tag:I-PERIOD

Token:से
Predict_Tag:o

Token:दिल्ली
Predict_Tag:o

Token:में
Predict_Tag:o

Token:रहता
Predict_Tag:o

Token:ह
Predict_Tag:o

Token:##ू
Predict_Tag:X

Token:##ं
Predict_Tag:X

Token:[SEP]
Predict_Tag:o

